<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:200%;
           font-family:Arial;letter-spacing:0.5px">

<p width = 20%, style="padding: 10px;
              color:white;">
SQL: Groupby and Joins
              
</p>
</div>

Data Science Cohort Live NYC Sept 2022
<p>Phase 1: Topic 7</p>
<br>
<br>

<div align = "right">
<img src="Images/flatiron-school-logo.png" align = "right" width="200"/>
</div>
    
   

In [ ]:
import pandas as pd
import sqlite3
import pandas as pd
conn = sqlite3.connect("data/flights.db")


We've seen aggregations last lecture:

In [ ]:
# Max value for longitude
pd.read_sql('''
    SELECT 
        -- Note we have to cast to a numerical value first
        MAX(CAST(longitude AS REAL))
        
    FROM 
        airports
''', conn)

In [ ]:
%%bash

sqlite3 data/flights.db
.schema airlines

In [ ]:
# Effectively counts all the active airlines 
pd.read_sql('''
    SELECT 
        COUNT(*) AS number_of_active_airlines
    FROM 
        airlines
    WHERE 
        active='Y'
''', conn)

This is OK, but want to:
- often compute aggregations across different groups 

#### Grouping in SQL

- Group by values of a categorical column
- Apply aggregation/transformation to groups

GROUP BY statement -- typically used with aggregation!



SELECT colwithgroups,<br>
       agg_function(another_col)<br>
FROM table<br>
GROUP BY colwithgroups<br>

In [ ]:
df_results = pd.read_sql('''
    SELECT *
    FROM 
        airlines
        LIMIT 5
''', conn)
df_results

In [ ]:
df_results = pd.read_sql('''
    SELECT 
        active, COUNT(*) AS number_of_airlines
    FROM 
        airlines
    GROUP BY
        active
''', conn)

df_results

Let's see another example, finding the airport with the highest altitude in each country:

In [ ]:
df_results = pd.read_sql('''
    SELECT 
        country,
        name as airport_name, MAX(CAST(altitude AS int)) AS altitude
    FROM 
        airports
    GROUP BY
        country
''', conn)

df_results

Let's look at the head of the airlines table:

In [ ]:
df_results = pd.read_sql('''
    SELECT *
    FROM 
        airlines 
    LIMIT 3
''', conn)

df_results

 #### Excercise 
Count the number of active flights grouped by country. Order the active flights from highest to lowest. 

<details>
    <summary><b><u>Possible Solution</u></b></summary>

```python 
df_results = pd.read_sql('''
    SELECT country, COUNT(active) Active_Flights
    FROM
        airlines
    WHERE active = 'Y'
    GROUP BY country
    ORDER BY Active_Flights DESC

''', conn)
df_results

```
</details>

#### Filtering Groups with HAVING

- Can filter columns/tables with WHERE
- WHERE doesn't work when filtering aggregations on groupby
- `HAVING` will do the trick. 

Let's only return countries with active airlines having a count greater than 30:

In [ ]:
pd.read_sql('''
    SELECT 
        country,
        COUNT() AS num
    FROM 
        airlines
    WHERE 
        active='Y'
    GROUP BY 
        country
    HAVING
        num > 30
    ORDER BY 
        num DESC
''', conn)

- Can use multiple aggregation functions in groupby.
- Filter on one of the aggregations.

##### Exercise 

Get the altitude of the highest airport in a country given that it has at least 100 airports.

<details>
    <summary><b><u>Possible Solution</u></b></summary>

```python 
pd.read_sql('''
    SELECT 
        country,
         COUNT(*) AS num_airports,
        MAX(CAST(altitude as int)) as max_altitude
       
    FROM 
        airports
    GROUP BY 
        country
    HAVING
        num_airports >= 100
''', conn)

```
</details>

**Exercise**

Get the number of airports of cities having more than one airport and the average airport altitude in the city.

Use GROUP BY and HAVING clauses for this.

<details>
    <summary><b><u>Possible Solution</u></b></summary>

```python 
pd.read_sql('''
SELECT city, COUNT(*) AS num_airports, AVG(CAST(altitude as int)) as average_altitude

FROM airports

GROUP BY city

HAVING num_airports > 1
''', conn)

```
</details>

#### Joins
- Saw this in Pandas
- Join information from two tables in a relational database together.
- For the two tables: can relate via a common key (aka foreign key).

SQL has many different kinds of joins. We only look at two:
- INNER JOIN
- LEFT JOIN 

A little picture:

<div>
<center><img src="Images/venn.png" align = "center" width="600"/></center>
</div>

#### Inner joins

- Join two tables together on a shared key. 
- Keep rows if the key is in both tables.



##### Syntax: Inner Join
SELECT
    table1.column_name, <br>
    table2.different_column_name <br>
FROM <br>
    table1 <br>
    INNER JOIN table2 <br>
        ON table1.shared_column_name = table2.shared_column_name <br>

Let's check out the airports and airline table. 

In [ ]:
pd.read_sql('''
    SELECT 
        *
    FROM
        routes
    LIMIT 5
    

''', conn)

In [ ]:
pd.read_sql('''
    SELECT 
        *
    FROM
        airlines
        LIMIT 5

''', conn)

The airline_id in the routes table and id column in airlines are common keys.

Can join these two tables keeping rows with keys in both tables.

- Note table aliasing

#### INNER JOIN

In [ ]:
pd.read_sql('''
    SELECT 
        al.name, al.callsign, rt.airline_id, rt.source, rt.dest
    FROM
        airlines AS al
        INNER JOIN
        routes AS rt
        ON al.id = rt.airline_id
        

''', conn)

#### LEFT JOIN 

Can join two tables keeping rows with keys in left table only.

In [ ]:
pd.read_sql('''
    SELECT 
        *
    FROM
        airlines as al
        
    LEFT JOIN routes as rt
            ON al.id = rt.airline_id
''', conn)

Can do it the other way:

In [ ]:
pd.read_sql('''
    SELECT 
        *
    FROM
        routes as rt
        
    LEFT JOIN airlines as al
            ON rt.airline_id = al.id 
''', conn)

A lot more unique entries on shared key in the airline table vs. the route table.

We won't go through RIGHT JOIN and FULL JOIN here -- its actually fairly self explanatory what they do.

#### CROSS JOIN

This can be pretty useful when you want to:
- compare every row in one table with every other row in another table.
- create all possible combinations of columns belonging in different tables.
    

A little picture might help:

<div>
<center><img src="images/cross_join.png" align = "center" width="600"/></center>
</div>

In [ ]:
pd.read_sql('''
    SELECT rt.airline_id as rt_id,
           rt.source as rt_source,
           rt.dest as rt_dest,
           al.id as al_id,
           al.name as al_name
           
    FROM
        routes as rt
        CROSS JOIN airlines as al
        
    LIMIT 10
''', conn)

Be careful: cross joins can take a long time!

#### SELF JOIN

This is not exactly a standard join operation. But it can be very useful.

- Join a table with itself via inner join.

**Use case:** 

Create pairwise comparisons for airports in the same country and sharing the same city.

Here, we do an inner join:

- Can join on multiple columns and conditions!!
- Self join is excellent use case for this.

In [ ]:
pd.read_sql('''
    SELECT ap1.country,
        ap1.city,
        ap1.name AS ap1_name, 
        ap1.code AS ap1_code,
        ap1.altitude as ap1_altitude,
        ap2.name AS ap2_name, 
        ap2.code AS ap2_code, 
        ap2.altitude as ap2_altitude
        
    FROM
        airports as ap1
        INNER JOIN airports  as ap2
            ON ap1.country = ap2.country
            AND ap1.city = ap2.city
            AND ap1.name <> ap2.name
''', conn)

#### Set Operations

- UNION: gets union, drops duplicates
- UNION ALL: gets union, keeps duplicates
- INTERSECT: gets intersection
- EXCEPT: gets what's in one table but not other.

Useful when you have two tables with same data scheme.

- Table1 has some fields with same kind of data as some fields in Table2.
- Can take intersection of fields from two tables with same kind of data, etc.
- Less flexible than joins, but fast and useful in many cases.

<div>
<center><img src="images/setopd.png" align = "center" width="600"/></center>
</div>

An example might be useful here:

Get countries in airports table not in airlines table.

Countries that have airports but no airlines.

In [ ]:
pd.read_sql('''
    SELECT ap.country 
    FROM airports as ap
    EXCEPT
    SELECT al.country
    FROM airlines as al
    LIMIT 10
''', conn)

Next up: subqueries!!

Can give us additional flexibility in accessing data!